In [86]:
pip install git+https://github.com/TolulopeOmo/Preprocess.git

  Cloning https://github.com/TolulopeOmo/Preprocess.git to /tmp/pip-req-build-2zd38rb9
  Running command git clone -q https://github.com/TolulopeOmo/Preprocess.git /tmp/pip-req-build-2zd38rb9
  Created wheel for Preprocess: filename=Preprocess-0.0.9-cp37-none-any.whl size=3537 sha256=28f2dbcd475f5eef46da13c322880613b4029432eae8612fd2762214ff217244
  Stored in directory: /tmp/pip-ephem-wheel-cache-tbg1n8lb/wheels/66/38/9a/bb27e62e0c6aa55f4c1e948c118472bbde4722a6964e0adcc5
Successfully built Preprocess


In [87]:
import pandas as pd
import numpy as np

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [89]:
from google.colab import files
uploaded = files.upload()

Saving imdb_reviews.txt to imdb_reviews (2).txt


In [90]:
import Preprocess as ps

In [91]:
df = pd.read_csv('imdb_reviews (1).txt', sep ='\t', header = None)

In [92]:
df.columns =['reviews', 'sentiments']

In [93]:
df.head()

,reviews,sentiments
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [94]:
df['reviews'] = df['reviews'].apply(lambda x: ps.cont_exp(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_special_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_accented_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_emails(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_html_tags(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_urls(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.make_base(x))

In [95]:
df['reviews'] = df['reviews'].apply(lambda x: str(x).lower())

In [96]:
df

,reviews,sentiments
0,a very very very slowmove aimless movie about ...,0
1,not sure who was more lose the flat character ...,0
2,attempt artiness with black white and clever c...,0
3,very little music or anything to speak of,0
4,the good scene in the movie was when gerardo i...,1
...,...,...
743,i just get bore watch jessice lange take her c...,0
744,unfortunately any virtue in this film producti...,0
745,in a word it is embarrassing,0
746,exceptionally bad,0


Data Preparation for Model Training

In [97]:
X = df['reviews']
y = df['sentiments']

In [98]:
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 22, stratify = y)

In [99]:
X_train.shape, X_test.shape

((598,), (150,))

ML Model Building

In [100]:
pipe = Pipeline([
                 ('tfidf', TfidfVectorizer( )),
                 ('clf', LogisticRegression(solver = 'liblinear'))
])

In [101]:
hyperparameters = {
    'tfidf__max_df' : (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__penalty': ('l2', 'l1'),
    'clf__C': (1, 2)

}

In [102]:
clf = GridSearchCV(pipe, hyperparameters , n_jobs =-1, cv = None)

In [103]:
%%time
clf.fit(X_train, y_train)

CPU times: user 1.95 s, sys: 88.9 ms, total: 2.04 s
Wall time: 24.1 s


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                      

In [104]:
clf.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=2, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1

In [105]:
clf.best_params_

{'clf__C': 2,
 'clf__penalty': 'l2',
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 1.0,
 'tfidf__ngram_range': (1, 2),
 'tfidf__use_idf': True}

In [106]:
clf.best_score_

0.7709383753501401

In [107]:
y_pred = clf.predict(X_test)

In [108]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.74      0.76        73
           1       0.76      0.79      0.78        77

    accuracy                           0.77       150
   macro avg       0.77      0.77      0.77       150
weighted avg       0.77      0.77      0.77       150



SVM Model

In [109]:
from sklearn.svm import LinearSVC

In [110]:
pipe = Pipeline([
                 ('tfidf', TfidfVectorizer( )),
                 ('clf', LinearSVC())
])

In [111]:
hyperparameters = {
    'tfidf__max_df' : (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__C': (1,2,2.5, 3)

}

In [112]:
clf = GridSearchCV(pipe, hyperparameters , n_jobs =-1, cv = 5)

In [113]:
%%time
clf.fit(X_train, y_train)

CPU times: user 1.8 s, sys: 52.8 ms, total: 1.85 s
Wall time: 22.9 s


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [114]:
clf.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=2, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                 

In [115]:
clf.best_params_

{'clf__C': 2,
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 0.5,
 'tfidf__ngram_range': (1, 2),
 'tfidf__use_idf': True}

In [116]:
clf.best_score_

0.7792577030812324

In [117]:
y_pred = clf.predict(X_test)

In [118]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.77      0.77        73
           1       0.78      0.79      0.79        77

    accuracy                           0.78       150
   macro avg       0.78      0.78      0.78       150
weighted avg       0.78      0.78      0.78       150



Model Testing and Saving

In [119]:
x = ['This is a great movie', 'I loved it']

In [120]:
clf.predict(x)

array([1, 1])

This shows that the first review is a positive one likewise the second review 

In [121]:
import pickle as pkl

In [122]:
pkl.dump(clf, open('model.pkl', 'wb'))